In [375]:
import numpy as np
import plotly.express as plx
import pandas as pd 

In [376]:
class NeuralNetwork:
    def __init__(self, input_nodes:int, hidden_nodes:int, output_nodes:int, learning_rate:float = 0.1):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate
        np.random.seed(110)
        self.hidden_weights = np.random.uniform(size=(self.input_nodes, self.hidden_nodes))
        self.hidden_bias =np.random.uniform(size=(1, self.hidden_nodes))
        self.output_weights = np.random.uniform(size=(self.hidden_nodes, self.output_nodes))
        self.output_bias = np.random.uniform(size=(1, self.output_nodes))
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self,x):
        return x * (1 - x)

    def forward_pass(self,inputs):
        hidden_layer_activation = np.dot(inputs, self.hidden_weights) + self.hidden_bias    
        hidden_layer_output = self.sigmoid(hidden_layer_activation)
    
        output_layer_activation = np.dot(hidden_layer_output, self.output_weights) + self.output_bias
        predicted_output = self.sigmoid(output_layer_activation)
        return predicted_output, hidden_layer_output

    def backward_pass(self,expected_output, predicted_output,hidden_layer_output)-> tuple[np.ndarray, np.ndarray, np.float64]:
        error = expected_output - predicted_output
        d_predicted_output = error * self.sigmoid_derivative(predicted_output)
        error_hidden_layer = d_predicted_output.dot(self.output_weights.T)
        d_hidden_layer = error_hidden_layer * self.sigmoid_derivative(hidden_layer_output) 
        sse = sum(error ** 2)
        return d_predicted_output, d_hidden_layer,sse
    
    
    def train(self,X:np.ndarray,y:np.ndarray,epochs:int=10):
        predicted_output = np.NAN
        SQE = []
        for epoch in range(epochs):
            predicted_output, hidden_layer_output = self.forward_pass(X)
            d_predicted_output, d_hidden_layer,error = self.backward_pass(y, predicted_output, hidden_layer_output)
            SQE.append(error) 
            
            self.output_weights += hidden_layer_output.T.dot(d_predicted_output) * self.learning_rate
            self.output_bias += np.sum(d_predicted_output, axis=0, keepdims=True) * self.learning_rate
            self.hidden_weights += X.T.dot(d_hidden_layer) * self.learning_rate
            self.hidden_bias += np.sum(d_hidden_layer, axis=0, keepdims=True) * self.learning_rate
        return predicted_output,SQE
    
    def predict(self, x_data:np.ndarray):
        predicted_output, hidden_layer_output = self.forward_pass(x_data)
        return np.round(predicted_output)
    
    def accuracy(self,X_act,Y_act):
        Y_pred = self.predict(X_act)
        lenght = len(X_act)
        return sum([1 / lenght for i in range(lenght) if Y_pred[i] == Y_act[i]])


In [377]:
inp_nod = 3
hid_nod = 2
out_nod = 1

X = np.array([
    np.array([0,0,0]),
    np.array([0,0,1]),
    np.array([0,1,1]),
    np.array([1,1,1]),
    np.array([1,0,0]),
    np.array([1,1,0]),
    np.array([0,1,0]),
    np.array([1,0,1]),
])
Y = np.array([[1],[0],[0],[1],[1],[0],[1],[0]])
epochs = 1000
model = NeuralNetwork(input_nodes=inp_nod,hidden_nodes=hid_nod,output_nodes=out_nod)
probka,SQE = model.train(X=X,y=Y,epochs=epochs)
dp = pd.DataFrame({"x":list(range(len(SQE))),"y":map(np.float64,SQE)})
min_sqe = min(dp["y"])
fig = plx.line(data_frame=dp,
               x="x",y="y",
               title="Изменение суммы квадратов ошибок с изменением эпохи при значении learning_rate = 0.1",
               labels={"x":"Эпоха","y":"SSE"})

fig.show()


In [378]:
X_test = np.array([
    np.array([0,0,0]),
    np.array([0,0,1]),
    np.array([0,1,1])
])
Y_test = np.array([[1],[0],[0]])
model.accuracy(X_test,Y_test)

1.0

In [379]:
epochs = 1000
SQE_lr = []
for lr in np.linspace(0,1,1000):
    model = NeuralNetwork(input_nodes=inp_nod,hidden_nodes=hid_nod,output_nodes=out_nod,learning_rate=lr)
    probka,SQE = model.train(X=X,y=Y,epochs=epochs)
    min_sqe = min(SQE)
    SQE_lr.append(min_sqe)
SQE_lr =list(map(np.float64,SQE_lr))
fig = plx.line(x=list(map(lambda x:x/1000,range(len(SQE_lr)))),
                      y=SQE_lr,
               title=f"Изменение суммы квадратов ошибок с изменением learning_rate при значении эпох {epochs}",
               labels={"x":"learning_rate","y":"SSE"})
fig.show()

C:\Users\isada\AppData\Local\Temp\ipykernel_18424\1379544823.py:8: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [380]:
min_SSE = min(SQE_lr)
print(min_SSE)

0.5273394995451224


In [383]:
fp = SQE_lr[0]
lr = 0.1
for i in range(1,len(SQE_lr)):
    delt = fp - SQE_lr[i]
    fp = SQE_lr[i]
    if delt < 0.001:
        lr = i/1000
        print(f"Самый оптимальный learning_rate это :{lr} \n При нем ошибка SSE:{SQE_lr[i]}")
        break

Самый оптимальный learning_rate это :0.323 
 При нем ошибка SSE:0.6844528929482275


In [385]:
epochs = 1000
model = NeuralNetwork(input_nodes=inp_nod,hidden_nodes=hid_nod,output_nodes=out_nod,learning_rate=lr)
probka,SQE = model.train(X=X,y=Y,epochs=epochs)
dp = pd.DataFrame({"x":list(range(len(SQE))),"y":map(np.float64,SQE)})
min_sqe = min(dp["y"])
fig = plx.line(data_frame=dp,
               x="x",y="y",
               title=f"Изменение суммы квадратов ошибок с изменением эпохи при значении learning_rate = {lr}",
               labels={"x":"Эпоха","y":"SSE"})

fig.show()